In [ ]:
## 인공지능을 위한 수학 1
## Term Project : Classifying Electronic Devices (8 classes)
## Custom Data Image Classification using PyTorch


## Module Import
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
import torch

import cv2
import numpy as np
import matplotlib.pyplot as plt

import glob
import PIL


## Google Drive Mount

from google.colab import drive
drive.mount('/content/drive')


## Trainig Parameter Setup

batch_size = 2
device = 'cuda' if cuda.is_available() else 'cpu'
max_epoch = 100
sub_fname = '12190369_b2_e100.txt'
val = True ## Change this to 'False' if you just want to try test images.


print(f'Training Custom Dataset on {device}\n{"="*44}')

## Train Dataset 구축

### Image/Label List 생성

all_image = glob.glob('/content/drive/MyDrive/samples/train/*/*.jpg')
all_image.sort

## 사용할 Image 파일들의 경로가 제대로 생성되었나 확인
#print(all_image)

## Image, Label List 설정
image_list=[]
label_list=[]


for item in all_image:
  image_list.append(cv2.imread(item))
  label_folder = item.split('/')[-2]
  label_list.append(int(label_folder))


### Train Tensor 생성 및 train_datset 객체 생성
all_image_tensor = torch.from_numpy((np.array(image_list).transpose(0,3,1,2)).astype(np.float32))
target_tensor = torch.tensor(np.array(label_list))

train_dataset = torch.utils.data.TensorDataset(all_image_tensor, target_tensor)

#print(type(train_dataset))


## Validation Dataset 구축

### Image/Label List 생성

if val:

  val_image = glob.glob('/content/drive/MyDrive/samples/train/*/*.jpg')
  val_image.sort

  val_image_list=[]
  val_label_list=[]


  for item in val_image:
    val_image_list.append(cv2.imread(item))
    label_folder = item.split('/')[-2]
    val_label_list.append(int(label_folder))

  val_image_list = image_list
  val_label_list = label_list


### Validation Tensor 생성 및 val_datset 객체 생성

  val_image_tensor = torch.from_numpy((np.array(val_image_list).transpose(0,3,1,2)).astype(np.float32))
  val_target_tensor = torch.tensor(np.array(val_label_list))



  # 클래스 개수에 따라 val_target_tensor의 크기를 (이미지 개수,)에서 (이미지 개수, 클래스 개수)로 변경
  val_dataset = torch.utils.data.TensorDataset(val_image_tensor, val_target_tensor)


## Test Dataset 파일 위치 지정
test_image = glob.glob('/content/drive/MyDrive/samples/test_all/*.jpg')
test_image.sort()

## Data loader
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=batch_size,
                               shuffle=True)
val_loader = data.DataLoader(dataset=val_dataset,
                              batch_size=batch_size,
                              shuffle=False)


## Network Architecture Definition

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l0 = nn.LayerNorm((3, 64, 64))
    self.l1 = nn.Conv2d(3, 16, 3, padding = 1)
    self.l2 = nn.Conv2d(16,16, 3, padding = 1)
    self.l3 = nn.Conv2d(16, 32, 3, padding = 1)
    self.l4 = nn.Linear(2048, 120)
    self.l5 = nn.Linear(120, 8)

   #torch.nn.init.xavier_normal_(self.l1.weight.data)

  def forward(self, x):
    x = self.l0(x)
    x = F.relu(self.l1(x))
    #print("1st size", x.size())
    x = F.max_pool2d(x, 2)
    #print("2nd size", x.size())
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.max_pool2d(x, 2)
    #print("3rd size", x.size())
    x = F.max_pool2d(x, 2)
    x = x.view(-1, 2048)
    x = F.relu(self.l4(x))
    return self.l5(x)

## Network Load


model = Net()
model.to(device)


## 특정 레이어의 웨이트를 출력하여 확인하고 싶을 때
#print(model.l1.weight)


## Training Setup

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)


## 모델 전체 weight parameter들의 이름과 값을 확인하고 싶을 때
#for param_tensor in model.state_dict():
#  print(param_tensor, "\t", model.state_dict()[param_tensor])



## Train Function

## 이 코드에서는 epoch 값을 인자로 받아 중간 결과 출력 시 사용.

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    # 10 iteration 마다 현재 loss값 출력
    if batch_idx % 10 == 0:
      print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
          epoch, batch_idx*len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))


## Test Function

## 매 epoch마다, test dataset에 대한 성능 측정

def val():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in val_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)

    # sum up batch loss
    test_loss += criterion(output, target).item()

    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(val_loader.dataset)
  print(f'==================\nValidation set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(val_loader.dataset)}'
        f'({100. * correct / len(val_loader.dataset):.0f}%)')




def test_submission():
  result_list = list(-1 for i in range(800))
  conf_list = list(-1 for i in range(800))

  model.eval()

  for item in test_image:

    img_name = int(item.split('/')[-1][:-4])
    print('Processing the image number', img_name, '.....')
    cur_tensor = torch.from_numpy((np.array(cv2.imread(item)).transpose(2,0,1)).astype(np.float32)).unsqueeze(0)
    cur_tensor = cur_tensor.to(device)
    output = model(cur_tensor)

    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    softmax_vec = torch.nn.functional.softmax(output.data, dim=1)
    #print(softmax_vec)
   # print(softmax_vec.size)
    conf = '%.05f'%softmax_vec[0][pred[0]]
  #  print(conf)
    result_list[img_name] = pred[0]
    conf_list[img_name] = conf

  mytxt = open(sub_fname, 'a')
  sub_cnt = 0

  for j in range(800):
    mytxt.write('%04d.jpg   %d   %s\n'%(sub_cnt, result_list[j], conf_list[j]))
    #mytxt.write('%04d.jpg   %d\n'%(sub_cnt, j[0]))
    sub_cnt+=1
  mytxt.close()


## Main

if __name__ == '__main__':

  ## time 모듈들은 각 epoch 마다 걸린 시간을 측정
  since = time.time()

  for epoch in range(1, max_epoch+1):
    epoch_start = time.time()

    train(epoch)

    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')

    if val:
      val()

    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Tesing time: {m:.0f}m {s:.0f}s')

  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

  print(f'Evaluation begins...')
  test_submission()
  print(f'Evaluation completed. The submittion file has been saved.')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Train Epoch : 75 | Batch Status : 1360/3200 (42%) | Loss : 0.000017
Train Epoch : 75 | Batch Status : 1380/3200 (43%) | Loss : 0.000188
Train Epoch : 75 | Batch Status : 1400/3200 (44%) | Loss : 0.000433
Train Epoch : 75 | Batch Status : 1420/3200 (44%) | Loss : 0.000470
Train Epoch : 75 | Batch Status : 1440/3200 (45%) | Loss : 0.001538
Train Epoch : 75 | Batch Status : 1460/3200 (46%) | Loss : 0.001385
Train Epoch : 75 | Batch Status : 1480/3200 (46%) | Loss : 0.000031
Train Epoch : 75 | Batch Status : 1500/3200 (47%) | Loss : 0.000001
Train Epoch : 75 | Batch Status : 1520/3200 (48%) | Loss : 0.000122
Train Epoch : 75 | Batch Status : 1540/3200 (48%) | Loss : 0.000054
Train Epoch : 75 | Batch Status : 1560/3200 (49%) | Loss : 0.000301
Train Epoch : 75 | Batch Status : 1580/3200 (49%) | Loss : 0.000063
Train Epoch : 75 | Batch Status : 1600/3200 (50%) | Loss : 0.000039
Train Epoch : 75 | Batch Status : 1620/3200 (51%) | Loss : 0.000009
Train Epoch 